In [1]:
%cd ../..

c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1


In [2]:
import os
import time
import shutil

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import joblib
pio.templates.default = "plotly_white"

import warnings
from pathlib import Path

import humanize

from sklearn.preprocessing import StandardScaler
from src.forecasting.ml_forecasting import (
    FeatureConfig,
    MissingValueConfig,
    MLForecast,
    ModelConfig,
    calculate_metrics,
)
from src.utils import plotting_utils
from src.utils.general import LogTime
from src.utils.ts_utils import metrics_adapter, forecast_bias, mae, mase, mse
from tqdm.autonotebook import tqdm
from src.forecasting.ml_forecasting import calculate_metrics
from src.utils import ts_utils
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

In [3]:
os.makedirs("imgs/chapter_13", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

## Utility Functions

In [4]:
def format_plot(fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        )
    )
    return fig

In [5]:
def evaluate_forecast(y_pred, test_target, train_target, model_name):
    metric_l = []
    for _id in tqdm(test_target.index.get_level_values(0).remove_unused_categories().categories, desc="Calculating metrics..."):
        target = test_target.xs(_id)
        _y_pred = y_pred.xs(_id)
        history = train_target.xs(_id)
        metric_l.append(
            calculate_metrics(target, _y_pred, name=model_name, y_train=history)
        )
    eval_metrics_df = pd.DataFrame(metric_l)
    agg_metrics = {
            "Algorithm": model_name,
            "MAE": ts_utils.mae(
                test_target, y_pred
            ),
            "MSE": ts_utils.mse(
                test_target, y_pred
            ),
            "meanMASE": eval_metrics_df.loc[:, "MASE"].mean(),
            "Forecast Bias": ts_utils.forecast_bias_aggregate(
                test_target, y_pred
            )
    }
    return agg_metrics, eval_metrics_df

In [6]:
from itertools import cycle


def plot_forecast(pred_df, forecast_columns, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [
        "rgba(" + ",".join([str(c) for c in plotting_utils.hex_to_rgb(c)]) + ",<alpha>)"
        for c in px.colors.qualitative.Plotly
    ]
    act_color = colors[0]
    colors = cycle(colors[1:])
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].energy_consumption,
            mode="lines",
            line=dict(color=act_color.replace("<alpha>", "0.9")),
            name="Actual Consumption",
        )
    )
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines",
                line=dict(dash="dot", color=next(colors).replace("<alpha>", "1")),
                name=display_col,
            )
        )
    return fig

def highlight_abs_min(s, props=''):
    return np.where(s == np.nanmin(np.abs(s.values)), props, '')

## Reading the data

In [7]:
try:
    #Reading the missing value imputed and train test split data
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    # Read in the Validation dataset as test_df so that we predict on it
    test_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    # test_df = pd.read_parquet(preprocessed/"block_0-7_test_missing_imputed_feature_engg.parquet")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

Selecting only half the blocks to make deep learning training smoother, faster and fit in GPU memory. This is purely a hardware constraint and done beacause dealing with the engineering challenges of handling huge datasets is out of scope for this book.

In [8]:
uniq_blocks = train_df.file.unique().tolist()
sel_blocks = sorted(uniq_blocks, key=lambda x: int(x.replace("block_","")))[:len(uniq_blocks)//2]
train_df = train_df.loc[train_df.file.isin(sel_blocks)]
test_df = test_df.loc[test_df.file.isin(sel_blocks)]
sel_lclids = train_df.LCLid.unique().tolist()

In [9]:
target = "energy_consumption"
index_cols = ["LCLid", "timestamp"]

In [10]:
# Setting the indices
train_df.set_index(index_cols, inplace=True, drop=False)
test_df.set_index(index_cols, inplace=True, drop=False)
# Creating a pred_df with actuals
pred_df = pd.concat([train_df[[target]], test_df[[target]]])

### Loading the GFM Forecast and calculating the aggregate metrics on selected LCLids

In [11]:
try:
    global_ml_fc_df = pd.read_pickle(output/"gfm_predictions_val_df.pkl")
    global_ml_fc_df = global_ml_fc_df.loc[global_ml_fc_df.index.get_level_values(0).isin(sel_lclids)]

    baseline_agg_metrics, baseline_metrics_df = evaluate_forecast(y_pred=global_ml_fc_df["GFM+Meta  (NativeLGBM)"], 
                                                     test_target = global_ml_fc_df["energy_consumption"], train_target = train_df["energy_consumption"], model_name="GFM+Meta  (NativeLGBM)")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Global Forecasting Models-ML.ipynb in Chapter10
    </div>
    """))

Calculating metrics...:   0%|          | 0/69 [00:00<?, ?it/s]

# Missing Value Handling

## Null check

In [12]:
nc = train_df.isnull().sum()
nc[nc>0]

pressure                                           1450
energy_consumption_lag_1                             69
energy_consumption_lag_2                            138
energy_consumption_lag_3                            207
energy_consumption_lag_4                            276
energy_consumption_lag_5                            345
energy_consumption_lag_46                          3174
energy_consumption_lag_47                          3243
energy_consumption_lag_48                          3312
energy_consumption_lag_49                          3381
energy_consumption_lag_50                          3450
energy_consumption_lag_334                        23046
energy_consumption_lag_335                        23115
energy_consumption_lag_336                        23184
energy_consumption_lag_337                        23253
energy_consumption_lag_338                        23322
energy_consumption_rolling_3_mean                   207
energy_consumption_rolling_3_std                

In [13]:
missing_value_config = MissingValueConfig(
    bfill_columns=[
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "energy_consumption_rolling_3_mean",
        "energy_consumption_rolling_3_std",
        "energy_consumption_rolling_6_mean",
        "energy_consumption_rolling_6_std",
        "energy_consumption_rolling_12_mean",
        "energy_consumption_rolling_12_std",
        "energy_consumption_rolling_48_mean",
        "energy_consumption_rolling_48_std",
        "energy_consumption_48_seasonal_rolling_3_mean",
        "energy_consumption_48_seasonal_rolling_3_std",
        "energy_consumption_336_seasonal_rolling_3_mean",
        "energy_consumption_336_seasonal_rolling_3_std",
        "energy_consumption_ewma__span_2880",
        "energy_consumption_ewma__span_336",
        "energy_consumption_ewma__span_48",
    ],
    ffill_columns=[],
    zero_fill_columns=[],
)

In [14]:
train_df = missing_value_config.impute_missing_values(train_df)
test_df = missing_value_config.impute_missing_values(test_df)

c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1\src\forecasting\ml_forecasting.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[bfill_columns] = df[bfill_columns].fillna(method="bfill")
c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1\src\forecasting\ml_forecasting.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[ffill_columns] = df[ffill_columns].fillna(method="ffill")
c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1\src\forecasting\ml_forecasting.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[bfill_columns] = df[bfill_columns].fillna(method="bfill")
c:\Users\tacke\OneDriv

In [15]:
nc = train_df.isnull().sum()
nc[nc>0]

Series([], dtype: int64)

# Running DL Global models

In [16]:
metric_record = [baseline_agg_metrics]

## PyTorch Tabular

In [17]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import FTTransformerConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

In [18]:
data_config = DataConfig(
    target=[target], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=[
        "visibility",
        "windBearing",
        "temperature",
        "dewPoint",
        "pressure",
        "apparentTemperature",
        "windSpeed",
        "humidity",
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "energy_consumption_rolling_3_mean",
        "energy_consumption_rolling_3_std",
        "energy_consumption_rolling_6_mean",
        "energy_consumption_rolling_6_std",
        "energy_consumption_rolling_12_mean",
        "energy_consumption_rolling_12_std",
        "energy_consumption_rolling_48_mean",
        "energy_consumption_rolling_48_std",
        "energy_consumption_48_seasonal_rolling_3_mean",
        "energy_consumption_48_seasonal_rolling_3_std",
        "energy_consumption_336_seasonal_rolling_3_mean",
        "energy_consumption_336_seasonal_rolling_3_std",
        "energy_consumption_ewma_span_2880",
        "energy_consumption_ewma_span_336",
        "energy_consumption_ewma_span_48",
        "timestamp_Elapsed",
        "timestamp_Month_sin_1",
        "timestamp_Month_sin_2",
        "timestamp_Month_sin_3",
        "timestamp_Month_sin_4",
        "timestamp_Month_sin_5",
        "timestamp_Month_cos_1",
        "timestamp_Month_cos_2",
        "timestamp_Month_cos_3",
        "timestamp_Month_cos_4",
        "timestamp_Month_cos_5",
        "timestamp_Hour_sin_1",
        "timestamp_Hour_sin_2",
        "timestamp_Hour_sin_3",
        "timestamp_Hour_sin_4",
        "timestamp_Hour_sin_5",
        "timestamp_Hour_cos_1",
        "timestamp_Hour_cos_2",
        "timestamp_Hour_cos_3",
        "timestamp_Hour_cos_4",
        "timestamp_Hour_cos_5",
        "timestamp_Minute_sin_1",
        "timestamp_Minute_sin_2",
        "timestamp_Minute_sin_3",
        "timestamp_Minute_sin_4",
        "timestamp_Minute_sin_5",
        "timestamp_Minute_cos_1",
        "timestamp_Minute_cos_2",
        "timestamp_Minute_cos_3",
        "timestamp_Minute_cos_4",
        "timestamp_Minute_cos_5",
        "timestamp_Is_quarter_end",
        "timestamp_Is_quarter_start",
        "timestamp_Is_year_end",
        "timestamp_Is_year_start",
        "timestamp_Is_month_start",
    ],
    categorical_cols=[
        "holidays",
        "precipType",
        "icon",
        "summary",
        "timestamp_Month",
        "timestamp_Quarter",
        "timestamp_WeekDay",
        "timestamp_Dayofweek",
        "timestamp_Dayofyear",
        "timestamp_Hour",
        "timestamp_Minute",
        "stdorToU", 
        "Acorn", 
        "Acorn_grouped", 
        "LCLid"
    ],
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=1000,
    accelerator="auto"
)
optimizer_config = OptimizerConfig()

In [19]:
# Making categorical columns as string because PyTorch Tabular currently doenst support categorical dtype
train_df[data_config.categorical_cols] = train_df[data_config.categorical_cols].astype(str)
test_df[data_config.categorical_cols] = test_df[data_config.categorical_cols].astype(str)

### FT Transformer Model

In [20]:
linear_head_config = LinearHeadConfig(
    layers="32",
    activation="ReLU"
)
model_config = FTTransformerConfig(
    task="regression",
    num_attn_blocks=3,
    num_heads=4,
    transformer_head_dim=64,
    attn_dropout=0.2,
    ff_dropout=0.1,
    learning_rate = 1e-3,
    head_config=linear_head_config.__dict__,
    metrics=["mean_squared_error"]
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2024-07-05 12:11:21,630 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

In [21]:
# tabular_model = TabularModel.load_from_checkpoint("notebooks/Chapter12/ft_transformer_global")

Warning - this next block takes a very long time. It can take upwards of 13 hours to run.

In [22]:
tabular_model.fit(train=train_df)
#Saving because you need not train the model again after restarting your kernel or machine etc. and can use the previous cell to load the saved model
tabular_model.save_model("notebooks/Chapter12/ft_transformer_global")
# Deleting automatically saved checkpoints
shutil.rmtree("saved_models")

Seed set to 42


2024-07-05 12:11:21,762 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-07-05 12:11:30,935 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_tabular\categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_tabular\categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the interme

2024-07-05 12:11:55,536 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: FTTransformerModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                  ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ FTTransformerBackbone │ 86.7 K │
│ 1 │ _embedding_layer │ Embedding2dLayer      │ 23.1 K │
│ 2 │ _head            │ LinearHead            │  1.1 K │
│ 3 │ loss             │ MSELoss               │      0 │
└───┴──────────────────┴───────────────────────┴────────┘

Trainable params: 110 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 110 K                                                                                                
Total estimated model params size (MB): 0

Output()

2024-07-06 01:44:40,985 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-07-06 01:44:40,991 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

In [23]:
forecast_df = tabular_model.predict(test_df, include_input_features=False)
pred_df = pred_df.join(forecast_df[["energy_consumption_prediction"]]).rename(
    columns={"energy_consumption_prediction": model_config._model_name}
)
agg_metrics, eval_metrics_df = evaluate_forecast(
    y_pred=forecast_df[f"{target}_prediction"],
    test_target=test_df["energy_consumption"],
    train_target=train_df["energy_consumption"],
    model_name=model_config._model_name,
)
metric_record.append(agg_metrics)

c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_tabular\categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\pytorch_tabular\categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the interme

Calculating metrics...:   0%|          | 0/69 [00:00<?, ?it/s]

# Evaluation of ML Forecast

In [24]:
agg_metrics_df = pd.DataFrame(metric_record)
agg_metrics_df.style.format({"MAE": "{:.4f}", 
                          "MSE": "{:.4f}", 
                          "meanMASE": "{:.4f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,GFM+Meta (NativeLGBM),0.0871,0.0334,1.0685,-0.54%
1,FTTransformerModel,0.0896,0.0338,1.1664,0.40%


In [25]:
metrics_df = pd.concat([eval_metrics_df,baseline_metrics_df])

In [26]:
fig = px.histogram(metrics_df, 
                   x="MASE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,2])
# fig.write_image("imgs/chapter_13/ft_mase_dist.png")
fig.show()

In [27]:
fig = px.histogram(metrics_df, 
                   x="MAE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
# fig.write_image("imgs/chapter_13/ft_mae_dist.png")
fig.update_layout(xaxis_range=[0,0.2])
fig.show()

In [28]:
fig = px.histogram(metrics_df, 
                   x="MSE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,0.05])
# fig.write_image("imgs/chapter_13/ft_mse_dist.png")
fig.show()

In [29]:
fig = px.histogram(metrics_df, 
                   x="Forecast Bias", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=250,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="Forecast Bias", ylabel="Probability Density", title="Distribution of Forecast Bias in the dataset")
fig.update_layout(xaxis_range=[-15,10])
# fig.write_image("imgs/chapter_13/ft_bias_dist.png")
fig.show()

# Saving the Forecasts and Metrics

In [30]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [31]:
pred_df.to_pickle(output/"ft_transformer_val_df.pkl")
metrics_df.to_pickle(output/"ft_transformer_metrics_val_df.pkl")
agg_metrics_df.to_pickle(output/"ft_transformer_aggregate_metrics_val.pkl")